https://ai.google.dev/gemini-api/tutorials/document_search

https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/gemini

In [3]:
%pip install rapidfuzz

Note: you may need to restart the kernel to use updated packages.


In [4]:
from google.cloud import bigquery
import re
import json
from datetime import datetime
import numpy as np
import pandas as pd
from difflib import SequenceMatcher
from rapidfuzz import fuzz, process

In [14]:
def read_bq():
    client = bigquery.Client()
    
    query = f"""
        SELECT *
        FROM `intern-project-415606.Criminal_Dataset.criminal_data`
    """
    
    try:
        query_job = client.query(query)
        df = query_job.to_dataframe()
        return df
    except Exception as e:
        print(f"Error: {e}")

In [15]:
df = read_bq()
df.head()

,JLR_LINK,TRANS_TYPE_OF_CASE,TRANS_LEGAL_RELATIONSHIP,PDF_TEXT,EXTRACT,ID,NAME,Year,Month,Day,GENDER,BIRTH
0,https://congbobanan.toaan.gov.vn/2ta827827t1cv...,採取行政處理措施的決定,放入強制性戒毒機構,<Page:1>TÒA ÁN NHÂN DÂN CỘNG HÒA XÃ HỘI CHỦ NG...,Họ và tên: Nguyễn Văn C – Giới tính: Nam Sinh...,No_Id,Nguyễn Văn C,1994,1,25,Male,1994-01-25 00:00:00
1,https://congbobanan.toaan.gov.vn/2ta889668t1cv...,刑事,犯罪非法持有毒品,<Page:1>TÒA ÁN NHÂN DÂN CỘNG HÒA XÃ HỘI CHỦ NG...,"Họ và tên: Trần Ngọc T, sinh ngày 08/12/1993....",No_Id,Trần Ngọc T,1993,12,8,Male,1993-12-08 00:00:00
2,https://congbobanan.toaan.gov.vn/2ta153804t1cv...,刑事,盜竊財產罪,<Page:1>TOÀ ÁN NHÂN DÂN CỘNG HOÀ XÃ HỘI CHỦ NG...,"Bùi Văn T, sinh ngày 09/12/1989 Tại: Xóm T – ...",No_Id,Bùi Văn T,1989,12,9,Male,1989-12-09 00:00:00
3,https://congbobanan.toaan.gov.vn/2ta1126341t1c...,刑事,犯罪非法持有毒品,<Page:1>TÒA ÁN NHÂN DÂN CỘNG HÒA XÃ HỘI CHỦ NG...,"Nguyễn Văn Đ, sinh ngày 04/8/1978 tại huyện H...",No_Id,Nguyễn Văn Đ,1978,8,4,Male,1978-08-04 00:00:00
4,https://congbobanan.toaan.gov.vn/2ta1066372t1c...,刑事,欺詐佔有財產罪,<Page:1>TÒA ÁN NHÂN DÂN CỘNG HÒA XÃ HỘI CHỦ NG...,"Cao Thị B, sinh ngày 01/01/1975, tại huyện C,...",No_Id,Cao Thị B,1975,1,1,Female,1975-01-01 00:00:00


In [22]:
a = df.loc[df['JLR_LINK'] == "https://congbobanan.toaan.gov.vn/2ta946636t1cvn/chi-tiet-ban-an", 'PDF_TEXT']
for b in a:
    print(b)
    break

<Page:1>TÒA ÁN NHÂN DÂN CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM
THÀNH PHỐ LẠNG SƠN Độc lập - Tự do - Hạnh phúc
TỈNH LẠNG SƠN
Bản án số: 28/2022/HS-ST
Ngày 07-4-2022
NHÂN DANH
NƯỚC CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM
TÒA ÁN NHÂN DÂN THÀNH PHỐ LẠNG SƠN, TỈNH LẠNG SƠN
- Thành phần Hội đồng xét xử sơ thẩm gồm có:
Thẩm phán Chủ tọa phiên tòa: Bà Nguyễn Minh Huyền
Các Hội thẩm nhân dân: Bà Ngô Thị Thục
Bà Dương Thị Thu Thủy
- Thư ký phiên tòa: Bà Hoàng Hồng Hạnh - Thư ký Tòa án nhân dân thành
phố Lạng Sơn, tỉnh Lạng Sơn.
- Đại diện Viện Kiểm sát nhân dân thành phố Lạng Sơn, tỉnh Lạng Sơn
tham gia phiên tòa: Bà Lê Thị Khanh - Kiểm sát viên.
Ngày 07 tháng 4 năm 2022 tại trụ sở Tòa án nhân dân thành phố Lạng Sơn,
tỉnh Lạng Sơn, xét xử sơ thẩm công khai vụ án hình sự sơ thẩm thụ lý số:
18/2022/TLST-HS ngày 28 tháng 01 năm 2022 theo Quyết định đưa vụ án ra xét xử
số: 29/2022/QĐXXST-HS ngày 17 tháng 3 năm 2022, Quyết định hoãn phiên tòa
số 09/2022/HSST-QĐ ngày 29 tháng 3 năm 2022, đối với các bị cáo:
1.

## format date

In [54]:
def convert_to_standard_date(date_str):
    if re.match(r'\d{4}-\d{2}-\d{2}', date_str):
        return date_str
    # Handling Vietnamese format "dd tháng mm năm yyyy"
    viet_date_match = re.match(r'(\d{1,2}) tháng (\d{1,2}) năm (\d{4})', date_str)
    if viet_date_match:
        day = viet_date_match.group(1).zfill(2)
        month = viet_date_match.group(2).zfill(2)
        year = viet_date_match.group(3)
        return f"{year}-{month}-{day}"
    
    # Handling "dd-mm-yyyy" format
    try:
        return datetime.strptime(date_str, '%d/%m/%Y').strftime('%Y-%m-%d')
    except ValueError:
        pass
    
    # Handling "yyyy-mm" format, assume day as 01
    try:
        return datetime.strptime(date_str, '%Y-%m').strftime('%Y-%m')
    except ValueError:
        pass
    
    # Handling formats like "yyyy"
    try:
        return datetime.strptime(date_str, '%Y').strftime('%Y')
    except ValueError:
        pass
    
    
    # Handling "no information" or any other text
    if date_str.lower in ['no', 'N/A', 'null']:
        return 'No information'
    
    # Default return None for unhandled cases
    return None

# Apply the function to the birthdate column
df.loc[:,'standardized_birthdate'] = df['birthdate'].apply(convert_to_standard_date)

# Display the DataFrame
df.head()

,extract_id,jlr_link,name,birthdate,gender,province_name,district_name,village_name,full_address,crime,jail,jail_duration,fine,fine_total,other_punishment,time,standardized_birthdate
0,5116,https://congbobanan.toaan.gov.vn/2ta1153558t1c...,Vi Đức T,2004-08-16,Male,Nghệ An,Con Cuông,ĐT,"Bản ĐT, xã LKh, huyện Con Cuông, tỉnh Nghệ An",no,no,no,no,no,no,2024-05-28 18:49:01+00:00,2004-08-16
1,5116,https://congbobanan.toaan.gov.vn/2ta1153558t1c...,Vi Văn H,1982,Male,Nghệ An,Con Cuông,ĐT,"Bản ĐT, xã LKh, huyện Con Cuông, tỉnh Nghệ An",no,no,no,no,no,no,2024-05-28 18:49:01+00:00,1982
2,5116,https://congbobanan.toaan.gov.vn/2ta1153558t1c...,Vi Thị B,1985,Female,Nghệ An,Con Cuông,ĐT,"Bản ĐT, xã LKh, huyện Con Cuông, tỉnh Nghệ An",no,no,no,no,no,no,2024-05-28 18:49:01+00:00,1985
3,5116,https://congbobanan.toaan.gov.vn/2ta1153558t1c...,Hoàng Thị Y,1991,Female,Nghệ An,ĐL,LD,"LD, xã LS, huyện ĐL, tỉnh Nghệ An",no,no,no,no,no,no,2024-05-28 18:49:01+00:00,1991
4,5116,https://congbobanan.toaan.gov.vn/2ta1153558t1c...,Đinh Văn H,1999,Male,Nghệ An,ĐL,Xóm 8,"Xóm 8, xã TS, huyện Đ L, tỉnh Nghệ An",no,no,no,no,no,no,2024-05-28 18:49:01+00:00,1999


In [55]:
df['standardized_birthdate']

0        2004-08-16
1              1982
2              1985
3              1991
4              1999
            ...    
11295          1967
11296          1973
11297          1965
11298          1964
11299    1975-01-01
Name: standardized_birthdate, Length: 11300, dtype: object

In [56]:
# Filter rows where length of 'standardized_birthdate' is less than 10
filtered_df = df[df['standardized_birthdate'].str.len() < 8]
filtered_df['standardized_birthdate'].unique()

array(['1982', '1985', '1991', '1999', '2006', '1960', '2002', '1989',
       '1977', '1980', '1995', '1979', '1987', '1983', '1959', '1963',
       '2001', '1962', '1961', '2007', '1994', '1958', '1993', '1996',
       '1966', '1974', '1973', '1967', '1972', '2003', '1998', '1988',
       '1970', '1976', '1957', '1964', '1971', '1965', '1997', '1992',
       '1953', '1942', '1978', '1990', '1986', '1969', '1952', '1955',
       '1654', '1968', '1981', '2017', '1975', '1984', '2000', '2012',
       '1970-07', '2004', '1956', '1938', '1937', '1949', '1950', '1939',
       '1947', '1946', '2009', '2022', '1954', '2018', '1945', '1951',
       '1940'], dtype=object)

In [57]:
filtered_df

,extract_id,jlr_link,name,birthdate,gender,province_name,district_name,village_name,full_address,crime,jail,jail_duration,fine,fine_total,other_punishment,time,standardized_birthdate
1,5116,https://congbobanan.toaan.gov.vn/2ta1153558t1c...,Vi Văn H,1982,Male,Nghệ An,Con Cuông,ĐT,"Bản ĐT, xã LKh, huyện Con Cuông, tỉnh Nghệ An",no,no,no,no,no,no,2024-05-28 18:49:01+00:00,1982
2,5116,https://congbobanan.toaan.gov.vn/2ta1153558t1c...,Vi Thị B,1985,Female,Nghệ An,Con Cuông,ĐT,"Bản ĐT, xã LKh, huyện Con Cuông, tỉnh Nghệ An",no,no,no,no,no,no,2024-05-28 18:49:01+00:00,1985
3,5116,https://congbobanan.toaan.gov.vn/2ta1153558t1c...,Hoàng Thị Y,1991,Female,Nghệ An,ĐL,LD,"LD, xã LS, huyện ĐL, tỉnh Nghệ An",no,no,no,no,no,no,2024-05-28 18:49:01+00:00,1991
4,5116,https://congbobanan.toaan.gov.vn/2ta1153558t1c...,Đinh Văn H,1999,Male,Nghệ An,ĐL,Xóm 8,"Xóm 8, xã TS, huyện Đ L, tỉnh Nghệ An",no,no,no,no,no,no,2024-05-28 18:49:01+00:00,1999
5,5116,https://congbobanan.toaan.gov.vn/2ta1153558t1c...,Vi Khánh H,2006,Male,Nghệ An,Con Cuông,ChĐ,"Thôn ChĐ, xã ChKh, huyện Con Cuông, tỉnh Nghệ An",no,no,no,no,no,no,2024-05-28 18:49:01+00:00,2006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11294,1430,https://congbobanan.toaan.gov.vn/2ta1066372t1c...,Phùng Phước A,1952,Male,Trà Vinh,C,H,"ấp H, xã P, huyện C, tỉnh Trà Vinh",Fraudulent appropriation of property,no,no,yes,"4,561,765",no,2024-05-25 02:59:50+00:00,1952
11295,1430,https://congbobanan.toaan.gov.vn/2ta1066372t1c...,Trần Thị L,1967,Female,Trà Vinh,C,H,"ấp H, xã P, huyện C, tỉnh Trà Vinh",Fraudulent appropriation of property,no,no,yes,"5,834,081",no,2024-05-25 02:59:50+00:00,1967
11296,1430,https://congbobanan.toaan.gov.vn/2ta1066372t1c...,Trần Thị Lệ Tr,1973,Female,Trà Vinh,C,H,"ấp H, xã P, huyện C, tỉnh Trà Vinh",Fraudulent appropriation of property,no,no,yes,"32,317,893",no,2024-05-25 02:59:50+00:00,1973
11297,1430,https://congbobanan.toaan.gov.vn/2ta1066372t1c...,Trần Thị O,1965,Female,Trà Vinh,C,H,"ấp H, xã P, huyện C, tỉnh Trà Vinh",Fraudulent appropriation of property,no,no,yes,"61,696,162",no,2024-05-25 02:59:50+00:00,1965


## filter out without crime

In [58]:
mask = ~(df[['crime', 'jail', 'jail_duration', 'fine', 'fine_total', 'other_punishment']].isin(['no', 'N/A', 'NO', 'No',
                                                                                                'No Action Taken',
                                                                                                'no information', 'No information',
                                                                                                'null', 'null', 'None',
                                                                                                 'No Crime', 'No crime',
                                                                                                 'No criminal charge',
                                                                                                 'No criminal record',
                                                                                                 'Not specified'])).all(axis=1)

# Filter the DataFrame using the mask
df = df[mask]

df.head()

,extract_id,jlr_link,name,birthdate,gender,province_name,district_name,village_name,full_address,crime,jail,jail_duration,fine,fine_total,other_punishment,time,standardized_birthdate
12,5354,https://congbobanan.toaan.gov.vn/2ta816509t1cv...,Nguyễn Đình T,1998-11-16,Male,Bắc Kạn,B,Đ,"Thôn Đ, thị trấn P, huyện B, tỉnh Bắc Kạn",Theft,yes,18 months,no,no,no,2024-05-29 11:43:55+00:00,1998-11-16
13,5354,https://congbobanan.toaan.gov.vn/2ta816509t1cv...,Nông Văn T,1995-02-07,Male,Bắc Kạn,B,T,"Thôn T, xã V, huyện B, tỉnh Bắc Kạn",Theft,yes,36 months,no,no,no,2024-05-29 11:43:55+00:00,1995-02-07
14,5359,https://congbobanan.toaan.gov.vn/2ta260824t1cv...,Doãn Dương D,1976-11-20,Male,Quảng Bình,B,Tâ,"thôn Tâ, xã C, huyện B, tỉnh Quảng Bình",Theft,no,no,no,no,15 months of non-custodial probation,2024-05-29 11:45:32+00:00,1976-11-20
15,5359,https://congbobanan.toaan.gov.vn/2ta260824t1cv...,Dương Đức Đ,1972-11-05,Male,Quảng Bình,B,3,"thôn 3, xã Đ, huyện B, tỉnh Quảng Bình",Theft,no,no,no,no,15 months of non-custodial probation,2024-05-29 11:45:32+00:00,1972-11-05
16,5282,https://congbobanan.toaan.gov.vn/2ta636354t1cv...,Nguyễn Thanh B,1986-05-25,Male,Lâm Đồng,TP. B,Phường X,"Số C đường T, Phường X, TP. B, tỉnh Lâm Đồng",Theft,yes,9 months,no,no,no,2024-05-29 09:25:54+00:00,1986-05-25


## format jail_duration

In [59]:
#filtered_df['jail_duration'].unique()

In [60]:
def convert_to_months(duration):
    # Define conversion rates
    year_to_month = 12
    month_to_month = 1

    # Dictionary for Vietnamese translations
    vietnamese_to_english = {
        'năm': 'year',
        'tháng': 'month',
        'chung thân': 'life imprisonment',
        'tù chung thân': 'life imprisonment',
        'mười hai': 'twelve',
        'Chín': 'nine',
        'Hai': 'two',
        'Sáu': 'six',
        'bảy': 'seven',
        'Một': 'one',
        '03': 'three',
        '06': 'six',
        '07': 'seven',
        '08': 'eight',
        '09': 'nine'
    }

    # Replace Vietnamese terms with English equivalents
    for viet, eng in vietnamese_to_english.items():
        duration = duration.replace(viet, eng)

    # Handle special cases
    if 'life' in duration.lower():
        return 'Life Imprisonment'
    if 'death' in duration.lower():
        return 'Death Sentence'
    if 'no' in duration.lower() or 'yes' in duration.lower():
        return 'Not Applicable'

    # Extract numbers and units
    numbers = re.findall(r'\d+', duration)
    units = re.findall(r'year|month', duration, re.IGNORECASE)

    # Convert all to months
    total_months = 0
    for number, unit in zip(numbers, units):
        if 'year' in unit.lower():
            total_months += int(number) * year_to_month
        elif 'month' in unit.lower():
            total_months += int(number) * month_to_month

    return total_months

df.loc[:, 'standardized_jail_duration'] = df['jail_duration'].apply(convert_to_months)

df['standardized_jail_duration'].unique()

array([18, 36, 'Not Applicable', 9, 12, 15, 0, 6, 39, 24, 3, 27, 96, 84,
       48, 60, 45, 81, 93, 30, 42, 7, 8, 11, 16, 13, 21, 72,
       'Life Imprisonment', 192, 126, 108, 204, 34, 5, 73, 4, 20, 14, 78,
       54, 26, 180, 32, 28, 132, 144, 31, 33, 120, 87, 10, 66, 90, 22,
       240, 216, 37, 162, 102, 17, 51, 156, 82, 19, 50, 360, 136, 63, 62,
       150, 46, 25, 57, 40, 38, 41, 107, 'Death Sentence', 88, 99, 174,
       186, 198, 69, 74, 29, 312, 252, 228, 141, 264, 123, 114, 105, 138,
       210, 23, 92, 348, 49, 303, 285, 145, 168, 110, 106, 147, 190, 43,
       86, 44, 52, 61, 297, 291, 35, 234, 68, 336, 249, 219, 129],
      dtype=object)

## format fine total

In [61]:
#df['fine_total'].unique()

In [86]:
def standardize_amount(amount):
    # Check for monthly payments and extract the number of months if present
    monthly_payment_match = re.search(r'(\d+([.,]\d+)*)\s*VND/month\s*for\s*(\d+)\s*months', amount, re.IGNORECASE)
    if monthly_payment_match:
        monthly_payment = monthly_payment_match.group(1).replace('.', '').replace(',', '')
        months = int(monthly_payment_match.group(3))
        return round(float(monthly_payment) * months)  # Return the total payment over the period
    
    # Handle ranges (e.g., "10.000.000-15.000.000")
    range_match = re.search(r'(\d+[.,\d]*)\s*[-đồng tođến]+\s*(\d+[.,\d]*)', amount, re.IGNORECASE)
    if range_match:
        low_amount = range_match.group(1).replace('.', '').replace(',', '')
        high_amount = range_match.group(2).replace('.', '').replace(',', '')
        # Convert to float and take the average of the range
        try:
            low_amount = float(low_amount)
            high_amount = float(high_amount)
            return round((low_amount + high_amount) / 2)
        except ValueError:
            return 0
        
    # Remove non-numeric characters but keep the decimal point and comma
    amount = re.sub(r'[^\d.,]', '', amount)
    # Determine if commas or periods are used as thousand separators or decimal points
    if ',' in amount and '.' in amount:
        if amount.find(',') < amount.find('.'):
            amount = amount.replace(',', '')
        else:
            amount = amount.replace('.', '')
            
    # Replace remaining commas with dots if they are used as decimal separators
    amount = amount.replace('.', ',')
    amount = amount.replace(',', '')
    # Convert to float and round to nearest integer (assuming no cents are needed)
    try:
        standardized_amount = eval(amount)
    except:
        standardized_amount = 0
    return standardized_amount

df.loc[:, 'standardized_fine_total'] = df['fine_total'].apply(standardize_amount)

df['standardized_fine_total'].unique()

array([         0,    5000000,     200000,   12000000,     770000,
          2500000,  149000000,  200000000,  182000000,   35000000,
         40000000,   32000000,   25000000,   28000000,   22000000,
         20000000,   10000000,   15000000,  100000000,   30000000,
          2700000,    7000000,    6000000,   60000000,     500000,
            50000,     950000,   23000000,   21000000,   50000000,
          1050000,    1000000,    1600000,       5000,    2000000,
          3800000,    1700000,     260000,   12500000,    1400000,
           800000,  778801000,     575000,    1500000,    1200000,
          9500000,    1070000,      80000,  228750000,  246540000,
        126000000,  210000000,     290000,     750000,    3000000,
         27000000,         12,   22792500,    2250000,    1800000,
          1350000,     370000,    9000000,    8000000,     894000,
         24000000,   68800000,    2870000,   11940000,    1690000,
          5700000,    6610000,         13,    4000000,     250

In [89]:
# Correct filtering condition
pdf = df[(df['standardized_fine_total'] < 1000) & (df['standardized_fine_total'] > 0)]
pdf
# # Iterate over the filtered DataFrame and print the results
# for index, row in pdf.iterrows():
#     print(row['fine_total'], row['standardized_fine_total'])


,extract_id,jlr_link,name,birthdate,gender,province_name,district_name,village_name,full_address,crime,jail,jail_duration,fine,fine_total,other_punishment,time,standardized_birthdate,standardized_jail_duration,standardized_fine_total
2787,3953,https://congbobanan.toaan.gov.vn/2ta558092t1cv...,Lại Thị Ngọc N,1980-05-12,Female,A,V,TH,"Khối TH, phường H, thành phố V, tỉnh A",Gambling,no,no,yes,12 months,no,2024-05-28 15:55:10+00:00,1980-05-12,Not Applicable,12
3037,4683,https://congbobanan.toaan.gov.vn/2ta801290t1cv...,Ma Văn C,1962-02-17,Male,Đ,K,X,"Thôn X, xã E, huyện K, tỉnh Đ",Gambling,no,no,yes,1 year 3 months,no,2024-05-28 17:44:56+00:00,1962-02-17,Not Applicable,13
3038,4683,https://congbobanan.toaan.gov.vn/2ta801290t1cv...,Ma Văn Đ,1986-11-25,Male,Đ,K,E,"Thôn E, xã E, huyện K, tỉnh Đ",Gambling,no,no,yes,1 year 3 months,no,2024-05-28 17:44:56+00:00,1986-11-25,Not Applicable,13
3039,4683,https://congbobanan.toaan.gov.vn/2ta801290t1cv...,Ma Văn L,1991-05-27,Male,Đ,K,X,"Thôn X, xã E, huyện K, tỉnh Đ",Gambling,no,no,yes,1 year 3 months,no,2024-05-28 17:44:56+00:00,1991-05-27,Not Applicable,13
3040,4683,https://congbobanan.toaan.gov.vn/2ta801290t1cv...,Ma Văn Nam,1987-05-12,Male,Đ,K,X,"Thôn X, xã E, huyện K, tỉnh Đ",Gambling,no,no,yes,1 year 3 months,no,2024-05-28 17:44:56+00:00,1987-05-12,Not Applicable,13
3041,4683,https://congbobanan.toaan.gov.vn/2ta801290t1cv...,Nguyễn Minh T,1978,Male,Đ,K,X,"Thôn X, xã E, huyện K, tỉnh Đ",Gambling,no,no,yes,1 year 3 months,no,2024-05-28 17:44:56+00:00,1978,Not Applicable,13
4154,5529,https://congbobanan.toaan.gov.vn/2ta1033019t1c...,Trần Thị N,1972,Female,Vĩnh Phúc,B,Đ,"Tổ dân phố Đ, thị trấn L, huyện B, tỉnh Vĩnh Phúc",Allowing unqualified person to drive a vehicle,no,no,yes,9 months non-custodial reform,no,2024-05-29 14:33:27+00:00,1972,Not Applicable,9
6508,600,https://congbobanan.toaan.gov.vn/2ta711148t1cv...,Phạm Tùng Đ,1992-08-19,Male,Thái Nguyên,thị xã P,"Tổ dân phố L, phường B","Tổ dân phố L, phường B, thị xã P, tỉnh Thái Ng...",False reporting,no,no,yes,15% of monthly income,Non-custodial reform for 12 months,2024-05-23 19:57:29+00:00,1992-08-19,Not Applicable,15
6509,600,https://congbobanan.toaan.gov.vn/2ta711148t1cv...,Đào Văn Ng,1973-07-02,Male,N,huyện V,Xã T,"Tổ dân phố T, phường P, thành phố S, tỉnh Thái...",False reporting,no,no,yes,10% of monthly income,"Non-custodial reform for 9 months, deducting 2...",2024-05-23 19:57:29+00:00,1973-07-02,Not Applicable,10
7396,2332,https://congbobanan.toaan.gov.vn/2ta687618t1cv...,Trần Mạnh Q,1996-05-14,Male,Phú Thọ,Thanh Sơn,L,"Khu D, xã L, huyện Thanh Sơn, tỉnh Phú Thọ",Illegal Detention,no,no,yes,24 months,Probation for 48 months,2024-05-26 18:25:39+00:00,1996-05-14,Not Applicable,24


## start comparison

In [11]:
name = "Vũ Văn Hùng"
birthdate = "1985-11-10"
province_name = "Bình Dương"
district_name = "Thủ Dầu Một"
village_name = "Phú Cường"

In [12]:
similarities_name = df['name'].apply(lambda x: fuzz.partial_ratio(name, x))
similarities_birthdate = df['birthdate'].apply(lambda x: fuzz.ratio(str(birthdate), str(x)))
similarities_province = df['province_name'].apply(lambda x: fuzz.ratio(province_name, x))
similarities_district = df['district_name'].apply(lambda x: fuzz.ratio(district_name, x))
similarities_village = df['village_name'].apply(lambda x:fuzz.ratio(village_name, x))

# Combine the similarities
#total_similarity = similarities_name + similarities_birthdate + similarities_province + similarities_district + similarities_village
total_similarity = similarities_name * 0.5 + similarities_birthdate * 0.1 + similarities_province * 0.2 + similarities_district * 0.1 + similarities_village * 0.1 

# Combine the similarities and original DataFrame
df['total_similarity'] = total_similarity

# Retrieve the top 10 most similar rows
top_10_rows = df.nlargest(10, 'total_similarity')

In [13]:
top_10_rows

,extract_id,jlr_link,name,birthdate,gender,province_name,district_name,village_name,full_address,crime,jail,jail_duration,fine,fine_total,other_punishment,time,standardized_birthdate,standardized_jail_duration,standardized_fine_total,total_similarity
1724,3983,https://congbobanan.toaan.gov.vn/2ta854842t1cv...,Vũ Văn H,1989-04-25,Male,Bình Dương,P,AT,"Ấp TB, xã AT, huyện P, tỉnh Bình Dương",Possession of Drugs,yes,1 year 3 months,no,no,no,2024-05-28 15:59:33+00:00,None,15,0,75.000000
953,4317,https://congbobanan.toaan.gov.vn/2ta66924t1cvn...,Vũ Văn H,2000-10-15,Male,Hải Dương,Thành phố H,Đinh Văn T,"số 22/88 Đinh Văn T, phường B, Thành phố H, tỉ...",Theft,no,no,no,no,10 months of non-custodial reform,2024-05-28 16:49:55+00:00,None,Not Applicable,0,73.425837
955,4317,https://congbobanan.toaan.gov.vn/2ta66924t1cvn...,Vũ Văn M,1999-12-09,Male,Hải Dương,tp H,Vũ Khâm L,"351 Vũ Khâm L, phường H, tp H, tỉnh Hải Dương",Theft,no,no,no,no,8 months of non-custodial reform,2024-05-28 16:49:55+00:00,None,Not Applicable,0,68.853801
8701,94,https://congbobanan.toaan.gov.vn/2ta566021t1cv...,Vũ Văn T,1980-10-04,Male,Hải Dương,C,khu dân cư số 8,"khu dân cư số 8, phường P, thành phố C, tỉnh H...",Gambling,yes,8 months,yes,"10,000,000",no,2024-05-23 16:04:40+00:00,None,8,10000000,68.798246
1176,4164,https://congbobanan.toaan.gov.vn/2ta258115t1cv...,Võ Văn L,1995-01-30,Male,Bình Dương,D,A,"Ấp A, xã C, huyện D, tỉnh Bình Dương",Drug use,no,no,no,no,Forced drug rehabilitation for 14 months at Cơ...,2024-05-28 16:27:27+00:00,None,Not Applicable,0,68.666667
535,6383,https://congbobanan.toaan.gov.vn/2ta203663t1cv...,Vũ Văn D,1991-01-10,Male,Hải Dương,T,Np,"Thôn A, xã Np, huyện T, tỉnh H",Gambling,no,no,no,no,9 months of probation,2024-05-29 20:12:18+00:00,None,Not Applicable,0,67.964912
4688,5510,https://congbobanan.toaan.gov.vn/2ta1065710t1c...,Tạ Văn T,1968-10-20,Male,Bình Dương,Thuận An,"số nhà 3/15C, tổ 3, khu phố B, phường B","số nhà 3/15C, tổ 3, khu phố B, phường B, thành...",Gambling,no,no,yes,20.000.000,no,2024-05-29 14:26:51+00:00,None,Not Applicable,20000000,67.460526
3431,6712,https://congbobanan.toaan.gov.vn/2ta910753t1cv...,Vũ Văn C,1983-09-02,Male,Hải Phòng,quận N,phường L,"Số 8/214 đường L, phường L, quận N, thành phố ...",Theft,yes,18 months,no,no,no,2024-05-30 10:09:19+00:00,None,18,0,67.217750
10245,1658,https://congbobanan.toaan.gov.vn/2ta858741t1cv...,Vũ Văn Đ,1997-01-13,Male,Quảng Ninh,thị xã Q,"Khu 8, phường Y","Khu 8, phường Y, thị xã Q, tỉnh Quảng Ninh",Tàng trữ trái phép chất ma túy,yes,18 months,no,no,no,2024-05-26 15:40:06+00:00,None,18,0,66.824561
2597,3798,https://congbobanan.toaan.gov.vn/2ta287161t1cv...,Vũ Văn H,2003-12-05,Male,Hải Dương,G,K,"thôn K, xã T, huyện G, tỉnh Hải Dương",Causing public disorder with rudimentary weapons,yes,12 months,no,no,no,2024-05-28 15:10:56+00:00,None,12,0,66.631579


In [91]:
from rapidfuzz import fuzz, process
from google.cloud import bigquery
from datetime import datetime
import pandas as pd
import numpy as np
import re

def read_bq():
    client = bigquery.Client()
    
    query = f"""
        SELECT *
        FROM `intern-project-415606.Criminal_Dataset.criminal_data_gemini`
    """
    
    try:
        query_job = client.query(query)
        df = query_job.to_dataframe()
        return df
    except Exception as e:
        print(f"Error: {e}")

# Function to convert various date formats to yyyy-mm-dd    
def convert_to_standard_date(date_str):
    if re.match(r'\d{4}-\d{2}-\d{2}', date_str):
        return date_str
    # Handling Vietnamese format "dd tháng mm năm yyyy"
    viet_date_match = re.match(r'(\d{1,2}) tháng (\d{1,2}) năm (\d{4})', date_str)
    if viet_date_match:
        day = viet_date_match.group(1).zfill(2)
        month = viet_date_match.group(2).zfill(2)
        year = viet_date_match.group(3)
        return f"{year}-{month}-{day}"
    
    # Handling "dd-mm-yyyy" format
    try:
        return datetime.strptime(date_str, '%d/%m/%Y').strftime('%Y-%m-%d')
    except ValueError:
        pass
    
    # Handling "yyyy-mm" format, assume day as 01
    try:
        return datetime.strptime(date_str, '%Y-%m').strftime('%Y-%m')
    except ValueError:
        pass
    
    # Handling formats like "yyyy"
    try:
        return datetime.strptime(date_str, '%Y').strftime('%Y')
    except ValueError:
        pass
    
    
    # Handling "no information" or any other text
    if date_str.lower in ['no', 'N/A', 'null']:
        return 'No information'
    
    # Default return None for unhandled cases
    return None

# Define the mask to filter out specific strings
def filter_specific_strings(df):
    mask = ~(df[['crime', 'jail', 'jail_duration', 'fine', 'fine_total', 'other_punishment']]
              .isin(['no', 'N/A', 'NO', 'No', 'No Action Taken', 'no information', 'No information', 'null', 'None', 
                     'No Crime', 'No crime', 'No criminal charge', 'No criminal record', 'Not specified'])).all(axis=1)
    # Filter the DataFrame using the mask
    df = df[mask]
    return df

# Function to convert jail duration to months
def convert_to_months(duration):
    # Define conversion rates
    year_to_month = 12
    month_to_month = 1

    # Dictionary for Vietnamese translations
    vietnamese_to_english = {
        'năm': 'year',
        'tháng': 'month',
        'chung thân': 'life imprisonment',
        'tù chung thân': 'life imprisonment',
        'mười hai': 'twelve',
        'Chín': 'nine',
        'Hai': 'two',
        'Sáu': 'six',
        'bảy': 'seven',
        'Một': 'one',
        '03': 'three',
        '06': 'six',
        '07': 'seven',
        '08': 'eight',
        '09': 'nine'
    }

    # Replace Vietnamese terms with English equivalents
    for viet, eng in vietnamese_to_english.items():
        duration = duration.replace(viet, eng)

    # Handle special cases
    if 'life' in duration.lower():
        return 'Life Imprisonment'
    if 'death' in duration.lower():
        return 'Death Sentence'
    if 'no' in duration.lower() or 'yes' in duration.lower():
        return 'Not Applicable'

    # Extract numbers and units
    numbers = re.findall(r'\d+', duration)
    units = re.findall(r'year|month', duration, re.IGNORECASE)

    # Convert all to months
    total_months = 0
    for number, unit in zip(numbers, units):
        if 'year' in unit.lower():
            total_months += int(number) * year_to_month
        elif 'month' in unit.lower():
            total_months += int(number) * month_to_month

    return total_months

def standardize_amount(amount):
    # Check for monthly payments and extract the number of months if present
    monthly_payment_match = re.search(r'(\d+([.,]\d+)*)\s*VND/month\s*for\s*(\d+)\s*months', amount, re.IGNORECASE)
    if monthly_payment_match:
        monthly_payment = monthly_payment_match.group(1).replace('.', '').replace(',', '')
        months = int(monthly_payment_match.group(3))
        return round(float(monthly_payment) * months)  # Return the total payment over the period
    
    # Handle ranges (e.g., "10.000.000-15.000.000")
    range_match = re.search(r'(\d+[.,\d]*)\s*[-đồng tođến]+\s*(\d+[.,\d]*)', amount, re.IGNORECASE)
    if range_match:
        low_amount = range_match.group(1).replace('.', '').replace(',', '')
        high_amount = range_match.group(2).replace('.', '').replace(',', '')
        # Convert to float and take the average of the range
        try:
            low_amount = float(low_amount)
            high_amount = float(high_amount)
            return round((low_amount + high_amount) / 2)
        except ValueError:
            return 0
        
    # Remove non-numeric characters but keep the decimal point and comma
    amount = re.sub(r'[^\d.,]', '', amount)
    # Determine if commas or periods are used as thousand separators or decimal points
    if ',' in amount and '.' in amount:
        if amount.find(',') < amount.find('.'):
            amount = amount.replace(',', '')
        else:
            amount = amount.replace('.', '')
            
    # Replace remaining commas with dots if they are used as decimal separators
    amount = amount.replace('.', ',')
    amount = amount.replace(',', '')
    # Convert to float and round to nearest integer (assuming no cents are needed)
    try:
        standardized_amount = eval(amount)
    except:
        standardized_amount = 0
    return standardized_amount

# Main function to process the DataFrame
def process_dataframe(df):
    # Apply the function to the birthdate column
    df.loc[:,'birthdate'] = df['birthdate'].apply(convert_to_standard_date)
    df.loc[:, 'standardized_jail_duration'] = df['jail_duration'].apply(convert_to_months)
    df.loc[:,'standardized_fine_total'] = df['fine_total'].apply(standardize_amount)
    df = filter_specific_strings(df)
    return df

def calculate_similarity(df, person):
    similarities_name = df['name'].apply(lambda x: fuzz.partial_ratio(person['name'], x))
    similarities_birthdate = df['birthdate'].apply(lambda x: fuzz.ratio(str(person['birthdate']), str(x)))
    similarities_province = df['province_name'].apply(lambda x: fuzz.ratio(person['province_name'], x))
    similarities_district = df['district_name'].apply(lambda x: fuzz.ratio(person['district_name'], x))
    similarities_village = df['village_name'].apply(lambda x:fuzz.ratio(person['village_name'], x))

    # Combine the similarities
    #total_similarity = similarities_name + similarities_birthdate + similarities_province + similarities_district + similarities_village
    total_similarity = similarities_name * 0.5 + similarities_birthdate * 0.1 + similarities_province * 0.2 + similarities_district * 0.1 + similarities_village * 0.1 

    # Combine the similarities and original DataFrame
    df['total_similarity'] = total_similarity

    # Retrieve the top 10 most similar rows
    top_10_rows = df.nlargest(10, 'total_similarity')  

    return top_10_rows

if __name__ == "__main__":
    # Load your DataFrame here
    df = read_bq()
    df = process_dataframe(df)
    person = {'name' : "Vũ Văn Hùng", 
              'birthdate' : "1985-11-10", 
              'province_name' : "Bình Dương", 
              'district_name' : "Thủ Dầu Một", 
              'village_name' : "Phú Cường"}
    
    top_10_rows = calculate_similarity(df, person)
    print(top_10_rows)

      extract_id                                           jlr_link      name  \
1816        3983  https://congbobanan.toaan.gov.vn/2ta854842t1cv...  Vũ Văn H   
1045        4317  https://congbobanan.toaan.gov.vn/2ta66924t1cvn...  Vũ Văn H   
508         6834  https://congbobanan.toaan.gov.vn/2ta1035760t1c...  Vũ Văn H   
1047        4317  https://congbobanan.toaan.gov.vn/2ta66924t1cvn...  Vũ Văn M   
9391          94  https://congbobanan.toaan.gov.vn/2ta566021t1cv...  Vũ Văn T   
1268        4164  https://congbobanan.toaan.gov.vn/2ta258115t1cv...  Võ Văn L   
590         6894  https://congbobanan.toaan.gov.vn/2ta823819t1cv...  Vũ Văn T   
4130        6383  https://congbobanan.toaan.gov.vn/2ta203663t1cv...  Vũ Văn D   
5654        5510  https://congbobanan.toaan.gov.vn/2ta1065710t1c...  Tạ Văn T   
3957        6712  https://congbobanan.toaan.gov.vn/2ta910753t1cv...  Vũ Văn C   

       birthdate gender province_name district_name  \
1816  1989-04-25   Male    Bình Dương             P  

In [92]:
import requests
import subprocess
import json
# Obtain the identity token using gcloud
result = subprocess.run(['gcloud', 'auth', 'print-identity-token'], stdout=subprocess.PIPE)
identity_token = result.stdout.decode('utf-8').strip()

# URL to which the POST request will be sent
url = "https://us-central1-intern-project-415606.cloudfunctions.net/comparison"

# Data to be sent in the POST request
data = {
    "person": {
        "name": "Nguyễn Thị Minh",
        "birthdate": "1990-03-22",
        "province_name": "Hà Nội",
        "district_name": "Hoàn Kiếm",
        "village_name": "Hàng Đào"
    }
}


# Headers including the Authorization token
headers = {
    "Authorization": f"bearer {identity_token}",
    "Content-Type": "application/json"
}

# Send the POST request
response = requests.post(url, json=data, headers=headers)

# Decode the JSON response to make it readable
decoded_response = response.json()

# Pretty print the JSON response
print(json.dumps(decoded_response, ensure_ascii=False, indent=4))

[
    {
        "extract_id": 6822,
        "jlr_link": "https://congbobanan.toaan.gov.vn/2ta921077t1cvn/chi-tiet-ban-an",
        "name": "Nguyễn Thị N",
        "birthdate": "1986-10-14",
        "gender": "Female",
        "province_name": "Hà Nội",
        "district_name": "Tây Hồ",
        "village_name": "Yên Phụ",
        "full_address": "Ngõ 128, phố Yên Phụ, phường Yên Phụ, quận Tây Hồ, thành phố Hà Nội",
        "crime": "Illegal possession of drugs",
        "jail": "yes",
        "jail_duration": "1 years 6 months 0 days",
        "fine": "no",
        "fine_total": "no",
        "other_punishment": "no",
        "time": 1717076521000,
        "standardized_jail_duration": 18,
        "standardized_fine_total": 0,
        "total_similarity": 76.8260869565
    },
    {
        "extract_id": 1025,
        "jlr_link": "https://congbobanan.toaan.gov.vn/2ta735413t1cvn/chi-tiet-ban-an",
        "name": "Nguyễn Thị L",
        "birthdate": "2004-10-10",
        "gender": "Female",